In [1]:
import datasets
import random

dataset_name = "natural_questions"
config_name = "default"
revision = "main"
split = None
max_train_samples = None
max_eval_samples = None
cache_dir = None

raw_datasets = datasets.load_dataset(
    dataset_name, config_name,
    revision=revision,
    split=split,
    cache_dir=cache_dir,
)

train_examples = raw_datasets["train"]
max_train_samples = max_train_samples
if max_train_samples is not None:
    # We will select sample from whole data if argument is specified
    train_examples = train_examples.select(range(max_train_samples))

print(f"Using {train_examples.num_rows} train examples.")

eval_examples = raw_datasets["validation"]
max_eval_samples = max_eval_samples
if max_eval_samples is not None:
    # We will select sample from whole data if argument is specified
    random_idxs = random.sample(range(len(eval_examples)), max_eval_samples)
    eval_examples = eval_examples.select(random_idxs)

print(f"Using {eval_examples.num_rows} eval examples.")

Using the latest cached version of the module from /dccstor/srosent3/huggingface_cache/modules/datasets_modules/datasets/natural_questions/da8124c83e3394df62c0f9bbc6c07652bbe9288ad833053134d5f0e978bb4ee5 (last modified on Tue Jun 20 10:13:49 2023) since it couldn't be found locally at natural_questions., or remotely on the Hugging Face Hub.
Reusing dataset natural_questions (/dccstor/srosent3/huggingface_cache/datasets/natural_questions/default/0.0.4/da8124c83e3394df62c0f9bbc6c07652bbe9288ad833053134d5f0e978bb4ee5)


  0%|          | 0/2 [00:00<?, ?it/s]

Using 307373 train examples.
Using 7830 eval examples.


In [4]:
def countX(lst, x):
    count = 0
    for ele in lst:
        if (ele == x):
            count = count + 1
    return count

def average_length(examples):
    short_answer_html_len  = 0
    short_answer_len = 0
    long_answer_html_len  = 0
    long_answer_len = 0
    short_answer_count = 0
    long_answer_count = 0
    index = 0
    for example in examples:
        index += 1
        if 'short_answers' in example['annotations']:
            for short_answer in example['annotations']['short_answers']:
                if len(short_answer['start_token']) == 0:
                    continue
                short_answer_count += 1
                tokens = example['document']['tokens']['token'][short_answer['start_token'][0]:short_answer['end_token'][0]]
                is_html = example['document']['tokens']['is_html'][short_answer['start_token'][0]:short_answer['end_token'][0]]
                short_answer_html_len += len(tokens)
                short_answer_len += countX(is_html, False)
        if 'long_answer' in example['annotations']:
            for long_answer in example['annotations']['long_answer']:
                if long_answer['candidate_index'] == -1:
                    continue
                long_answer_count += 1
                tokens = example['document']['tokens']['token'][long_answer['start_token']:long_answer['end_token']]
                is_html = example['document']['tokens']['is_html'][long_answer['start_token']:long_answer['end_token']]
                long_answer_html_len += len(tokens)
                long_answer_len += countX(is_html, False)
        if index % 5000 == 0:
            print(f'short: {short_answer_count} long: {long_answer_count} all: {index}')
            print(f'short answer w/html: {short_answer_html_len/short_answer_count}')
            print(f'short answer: {short_answer_len/short_answer_count}')
            print(f'long answer w/html: {long_answer_html_len/long_answer_count}')
            print(f'long answer: {long_answer_len/long_answer_count}')

average_length(train_examples)

short: 341 long: 468 all: 1000
short answer w/html: 5.296187683284457
short answer: 5.296187683284457
long answer w/html: 279.77777777777777
long answer: 203.5940170940171
short: 674 long: 948 all: 2000
short answer w/html: 4.640949554896142
short answer: 4.640949554896142
long answer w/html: 348.1044303797468
long answer: 239.14240506329114
short: 1035 long: 1438 all: 3000
short answer w/html: 4.370048309178744
short answer: 4.370048309178744
long answer w/html: 346.9777468706537
long answer: 249.1307371349096
short: 1360 long: 1953 all: 4000
short answer w/html: 4.29264705882353
short answer: 4.285294117647059
long answer w/html: 355.6912442396313
long answer: 253.38402457757297
short: 1708 long: 2455 all: 5000
short answer w/html: 4.1703747072599535
short answer: 4.164519906323185
long answer w/html: 358.24236252545825
long answer: 251.6826883910387
short: 2063 long: 2964 all: 6000
short answer w/html: 4.234609791565681
short answer: 4.229762481822588
long answer w/html: 352.3890013

KeyboardInterrupt: 

In [5]:
train_examples

In [ ]:
from primeqa.mrc.processors.preprocessors.natural_questions import NaturalQuestionsPreProcessor

from transformers import AutoConfig, AutoTokenizer
model_name = 'roberta-base'

config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=cache_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    use_fast=True,
    config=config,
)

preprocessor = NaturalQuestionsPreprocessor(
    stride=128,
    tokenizer=tokenizer,
)

# Train Feature Creation
train_examples, train_dataset = preprocessor.process_train(train_examples)

print(f"Preprocessing produced {train_dataset.num_rows} train features from {train_examples.num_rows} examples.")

# Validation Feature Creation
eval_examples, eval_dataset = preprocessor.process_eval(eval_examples)

print(f"Preprocessing produced {eval_dataset.num_rows} eval features from {eval_examples.num_rows} examples.")

ImportError: cannot import name 'NaturalQuestionsPreprocessor' from 'primeqa.mrc.processors.preprocessors.natural_questions' (/dccstor/srosent2/primeqa/primeqa/primeqa/mrc/processors/preprocessors/natural_questions.py)

In [ ]:
# compute length of answer span and passage answer